In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Conv2D, Dropout, MaxPool2D, Flatten
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score, r2_score
import numpy as np

Using TensorFlow backend.


In [47]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)
submission = pd.read_csv('sample_submission.csv', index_col=0)

In [6]:
train.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            9990, 9991, 9992, 9993, 9994, 9995, 9996, 9997, 9998, 9999],
           dtype='int64', name='id', length=10000)

In [7]:
train # rho 1개 src, dst 각각 35개씩 존재 # 근데이거 함수형으로 하면 잘돌아갈삘인디...

test

,rho,650_src,660_src,670_src,680_src,690_src,700_src,710_src,720_src,730_src,...,940_dst,950_dst,960_dst,970_dst,980_dst,990_dst,hhb,hbo2,ca,na
id,,,,,,,,,,,,,,,,,,,,,
0,25,0.37950,0.42993,0.52076,0.57166,0.67818,0.75476,0.83580,0.93623,0.96333,...,NaN,0.000000e+00,NaN,1.067504e-18,5.998949e-18,4.378513e-17,5.59,4.32,8.92,4.29
1,10,0.00000,0.00000,0.01813,0.00000,0.00000,0.01974,0.00321,0.00000,0.00000,...,1.343132e-08,6.112685e-09,2.130547e-09,NaN,9.710091e-09,NaN,0.00,2.83,7.25,4.64
2,25,0.00000,0.03289,0.02416,0.03610,0.05843,0.09015,0.14944,0.18578,0.25584,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.329725e-18,NaN,10.64,3.00,8.40,5.16
3,10,0.27503,0.31281,0.32898,0.41041,0.46587,0.52769,0.64369,0.73562,0.79865,...,2.245998e-10,1.299511e-10,7.782625e-11,NaN,4.088921e-10,NaN,5.67,4.01,5.05,4.35
4,15,1.01521,1.00872,0.98930,0.98874,1.01773,1.01632,1.00009,0.98217,1.01564,...,1.457955e-13,8.769053e-14,NaN,1.330237e-13,NaN,NaN,11.97,4.41,10.78,2.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,15,0.23929,0.30265,0.39929,0.51000,0.64072,0.77328,0.86722,0.95891,0.98998,...,6.788642e-16,0.000000e+00,5.516467e-16,9.690979e-16,1.391635e-15,5.460702e-14,12.68,4.11,12.31,0.10
9996,20,0.02583,0.00946,0.03650,0.01380,0.04093,0.04363,0.03260,0.05282,0.04608,...,1.814274e-16,1.117148e-16,7.352416e-17,NaN,5.825347e-16,4.226436e-15,8.46,4.11,10.46,3.12
9997,10,0.57589,0.62976,0.70571,0.77248,0.85402,0.92796,0.97691,0.98933,1.00493,...,4.828451e-11,3.110419e-11,8.119095e-12,NaN,1.504115e-10,4.003657e-10,9.84,3.20,10.45,2.06


In [48]:
train = train.interpolate()
test = test.interpolate()

train = train.fillna(train.mean())
test = test.fillna(train.mean())

print(train.shape)
print(test.shape)

(10000, 75)
(10000, 71)


In [49]:
train = np.array(train)
x_predict = np.array(test)

x = train[:,:71]
y = train[:,71:]

In [50]:
# 전처리
from sklearn.preprocessing import RobustScaler, StandardScaler
scaler = RobustScaler()
scaler.fit(x) 
x = scaler.transform(x)
x_predict = scaler.transform(x_predict)

# from sklearn.decomposition import PCA
# pca = PCA(n_components=12)
# pca.fit(x)
# x = pca.transform(x)
# x_predict = pca.transform(x_predict)

In [167]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler.fit(x) 
# x = scaler.transform(x)
# x_predict = scaler.transform(x_predict)

In [14]:
from sklearn.model_selection import train_test_split, GridSearchCV
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=43)
parameters = [
    {"n_estimators":[50,100,150,300] #"learning_rate":[0.1, 0.3, 0.01, 0.005, 0.001] # 이거 두개가 사실상 영향 미침
#     "max_depth":[3,5,7,20,50,3,4,5,100,150], "colsample_bylevel":[0.6,0.8,0.9,1],"colsample_bytree":[0.6,0.7,0.8,0.9,1]},
    } ]

model = GridSearchCV(RandomForestRegressor(),parameters,cv=5,n_jobs=1)
model.fit(x_train, y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('랜덤포레스트 mae 는', mae)

print('============================================')
print('best_estimator_',model.best_estimator_)
print('============================================')
print('best_params_',model.best_params_)
print('============================================')

C:\Users\bitcamp\anaconda3\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
C:\Users\bitcamp\anaconda3\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average

C:\Users\bitcamp\anaconda3\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
C:\Users\bitcamp\anaconda3\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average

랜덤포레스트 mae 는 1.4556918791666673
best_estimator_ RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=300, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)
best_params_ {'n_estimators': 300}


In [16]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('랜덤포레스트 mae 는', mae)

랜덤포레스트 mae 는 1.4548802541666659


In [19]:
train = pd.read_csv('train.csv', index_col=0)
x = train.iloc[:,:71]

In [20]:
feature_importances_ = model.feature_importances_
feature_importances = pd.DataFrame({'랜덤포레스트_feature_importance':feature_importances_}, x.columns)
feature_importances = feature_importances.sort_values(by=['랜덤포레스트_feature_importance'],ascending=False)
feature_importances

,랜덤포레스트_feature_importance
780_dst,0.048511
790_dst,0.043477
rho,0.041999
810_dst,0.041614
850_dst,0.029510
...,...
720_src,0.006048
700_src,0.006016
670_src,0.005939
690_src,0.005825


In [42]:
feature_importances[:50]

,랜덤포레스트_feature_importance
780_dst,0.048511
790_dst,0.043477
rho,0.041999
810_dst,0.041614
850_dst,0.029510
800_dst,0.027714
970_dst,0.026122
820_dst,0.025034
770_dst,0.024343
830_dst,0.023220


In [23]:
train

,rho,650_src,660_src,670_src,680_src,690_src,700_src,710_src,720_src,730_src,...,940_dst,950_dst,960_dst,970_dst,980_dst,990_dst,hhb,hbo2,ca,na
id,,,,,,,,,,,,,,,,,,,,,
0,25,0.37950,0.42993,0.52076,0.57166,0.67818,0.75476,0.83580,0.93623,0.96333,...,NaN,0.000000e+00,NaN,1.067504e-18,5.998949e-18,4.378513e-17,5.59,4.32,8.92,4.29
1,10,0.00000,0.00000,0.01813,0.00000,0.00000,0.01974,0.00321,0.00000,0.00000,...,1.343132e-08,6.112685e-09,2.130547e-09,NaN,9.710091e-09,NaN,0.00,2.83,7.25,4.64
2,25,0.00000,0.03289,0.02416,0.03610,0.05843,0.09015,0.14944,0.18578,0.25584,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.329725e-18,NaN,10.64,3.00,8.40,5.16
3,10,0.27503,0.31281,0.32898,0.41041,0.46587,0.52769,0.64369,0.73562,0.79865,...,2.245998e-10,1.299511e-10,7.782625e-11,NaN,4.088921e-10,NaN,5.67,4.01,5.05,4.35
4,15,1.01521,1.00872,0.98930,0.98874,1.01773,1.01632,1.00009,0.98217,1.01564,...,1.457955e-13,8.769053e-14,NaN,1.330237e-13,NaN,NaN,11.97,4.41,10.78,2.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,15,0.23929,0.30265,0.39929,0.51000,0.64072,0.77328,0.86722,0.95891,0.98998,...,6.788642e-16,0.000000e+00,5.516467e-16,9.690979e-16,1.391635e-15,5.460702e-14,12.68,4.11,12.31,0.10
9996,20,0.02583,0.00946,0.03650,0.01380,0.04093,0.04363,0.03260,0.05282,0.04608,...,1.814274e-16,1.117148e-16,7.352416e-17,NaN,5.825347e-16,4.226436e-15,8.46,4.11,10.46,3.12
9997,10,0.57589,0.62976,0.70571,0.77248,0.85402,0.92796,0.97691,0.98933,1.00493,...,4.828451e-11,3.110419e-11,8.119095e-12,NaN,1.504115e-10,4.003657e-10,9.84,3.20,10.45,2.06


In [25]:
train.iloc[:,71:]

,hhb,hbo2,ca,na
id,,,,
0,5.59,4.32,8.92,4.29
1,0.00,2.83,7.25,4.64
2,10.64,3.00,8.40,5.16
3,5.67,4.01,5.05,4.35
4,11.97,4.41,10.78,2.42
...,...,...,...,...
9995,12.68,4.11,12.31,0.10
9996,8.46,4.11,10.46,3.12
9997,9.84,3.20,10.45,2.06


In [26]:
corr

,rho,650_src,660_src,670_src,680_src,690_src,700_src,710_src,720_src,730_src,...,900_dst,910_dst,920_dst,930_dst,940_dst,950_dst,960_dst,970_dst,980_dst,990_dst
rho,1.000000,0.008473,0.007505,0.005690,0.003602,0.001866,-0.000208,-0.001967,-0.003442,-0.003734,...,-0.283605,-0.273559,-0.260415,-0.263783,-0.263486,-0.252727,-0.206371,-0.258663,-0.301847,-0.302813
650_src,0.008473,1.000000,0.993379,0.977147,0.950477,0.913308,0.865969,0.809923,0.750453,0.689206,...,-0.015461,-0.005535,-0.022289,-0.011971,-0.016962,0.006294,0.006598,0.030713,0.050650,0.082230
660_src,0.007505,0.993379,1.000000,0.993103,0.975988,0.946594,0.904078,0.849914,0.789711,0.725581,...,-0.026533,-0.016808,-0.031976,-0.022823,-0.027294,-0.005260,-0.001993,0.017931,0.034583,0.064443
670_src,0.005690,0.977147,0.993103,1.000000,0.992893,0.973548,0.938573,0.888876,0.830038,0.764550,...,-0.037280,-0.028143,-0.042109,-0.034449,-0.038518,-0.017450,-0.011891,0.003938,0.017131,0.045940
680_src,0.003602,0.950477,0.975988,0.992893,1.000000,0.991865,0.967757,0.926028,0.871532,0.807029,...,-0.046703,-0.037865,-0.051491,-0.044816,-0.048741,-0.028835,-0.020930,-0.008591,0.000874,0.028995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950_dst,-0.252727,0.006294,-0.005260,-0.017450,-0.028835,-0.039776,-0.048274,-0.058238,-0.065686,-0.071243,...,0.694102,0.705662,0.738809,0.865470,0.923559,1.000000,0.927350,0.927243,0.895577,0.769089
960_dst,-0.206371,0.006598,-0.001993,-0.011891,-0.020930,-0.029643,-0.037106,-0.044772,-0.051207,-0.055775,...,0.558994,0.529568,0.540947,0.683721,0.765355,0.927350,1.000000,0.926755,0.830797,0.642248
970_dst,-0.258663,0.030713,0.017931,0.003938,-0.008591,-0.020702,-0.031352,-0.041812,-0.050191,-0.056644,...,0.657734,0.665677,0.639056,0.772544,0.839336,0.927243,0.926755,1.000000,0.941833,0.829109
980_dst,-0.301847,0.050650,0.034583,0.017131,0.000874,-0.013969,-0.027064,-0.039717,-0.050204,-0.058698,...,0.656247,0.676473,0.715688,0.797750,0.849101,0.895577,0.830797,0.941833,1.000000,0.973048


In [30]:
corr

,rho,650_src,660_src,670_src,680_src,690_src,700_src,710_src,720_src,730_src,...,940_dst,950_dst,960_dst,970_dst,980_dst,990_dst,hhb,hbo2,ca,na
rho,1.000000,0.008473,0.007505,0.005690,0.003602,0.001866,-0.000208,-0.001967,-0.003442,-0.003734,...,-0.263486,-0.252727,-0.206371,-0.258663,-0.301847,-0.302813,-0.006255,-0.001448,0.005716,0.011872
650_src,0.008473,1.000000,0.993379,0.977147,0.950477,0.913308,0.865969,0.809923,0.750453,0.689206,...,-0.016962,0.006294,0.006598,0.030713,0.050650,0.082230,-0.000232,0.002426,-0.000594,0.011230
660_src,0.007505,0.993379,1.000000,0.993103,0.975988,0.946594,0.904078,0.849914,0.789711,0.725581,...,-0.027294,-0.005260,-0.001993,0.017931,0.034583,0.064443,0.000654,-0.000301,-0.000007,0.012373
670_src,0.005690,0.977147,0.993103,1.000000,0.992893,0.973548,0.938573,0.888876,0.830038,0.764550,...,-0.038518,-0.017450,-0.011891,0.003938,0.017131,0.045940,-0.000167,-0.002234,0.000615,0.012878
680_src,0.003602,0.950477,0.975988,0.992893,1.000000,0.991865,0.967757,0.926028,0.871532,0.807029,...,-0.048741,-0.028835,-0.020930,-0.008591,0.000874,0.028995,0.001064,-0.006298,0.000973,0.012947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990_dst,-0.302813,0.082230,0.064443,0.045940,0.028995,0.012819,-0.001383,-0.014680,-0.026358,-0.035586,...,0.804516,0.769089,0.642248,0.829109,0.973048,1.000000,-0.118116,-0.094332,-0.080156,-0.021809
hhb,-0.006255,-0.000232,0.000654,-0.000167,0.001064,0.001138,-0.000893,-0.001956,-0.002355,-0.001177,...,-0.180821,-0.142379,-0.097573,-0.102260,-0.122089,-0.118116,1.000000,-0.005313,0.016263,0.009942
hbo2,-0.001448,0.002426,-0.000301,-0.002234,-0.006298,-0.010138,-0.013323,-0.016297,-0.019541,-0.022108,...,-0.077260,-0.072727,-0.064166,-0.076454,-0.085435,-0.094332,-0.005313,1.000000,-0.002307,0.000726
ca,0.005716,-0.000594,-0.000007,0.000615,0.000973,0.003707,0.006024,0.008321,0.009084,0.009309,...,-0.099967,-0.146475,-0.162807,-0.157324,-0.110504,-0.080156,0.016263,-0.002307,1.000000,-0.005959


In [39]:
corr = train.corr()
Survived = corr.loc[:,['hhb','hbo2','ca','na']]
Survived = abs(Survived)
a = Survived.sort_values(by=['hhb','hbo2','ca','na'],
                              ascending=False)
# a = a[2:]
a.iloc[:50,:]

,hhb,hbo2,ca,na
hhb,1.000000,0.005313,0.016263,0.009942
870_dst,0.226053,0.092667,0.078517,0.002131
810_dst,0.214873,0.083051,0.060858,0.009175
820_dst,0.213707,0.080626,0.067721,0.017376
800_dst,0.213258,0.070051,0.058933,0.012515
900_dst,0.212781,0.084500,0.064433,0.077676
880_dst,0.207634,0.088777,0.076369,0.024746
890_dst,0.204140,0.070451,0.074633,0.085807
790_dst,0.203357,0.057963,0.063799,0.016859
780_dst,0.200927,0.044636,0.062192,0.022877


In [40]:
corr = train.corr()
Survived = corr.loc[:,['hhb']]
Survived = abs(Survived)
a = Survived.sort_values(by=['hhb'],
                              ascending=False)
a = a[2:]
a.iloc[:50,:]

,hhb
810_dst,0.214873
820_dst,0.213707
800_dst,0.213258
900_dst,0.212781
880_dst,0.207634
890_dst,0.204140
790_dst,0.203357
780_dst,0.200927
830_dst,0.200368
860_dst,0.198337


In [41]:
corr = train.corr()
Survived = corr.loc[:,['hbo2']]
Survived = abs(Survived)
a = Survived.sort_values(by=['hbo2'],
                              ascending=False)
a = a[2:]
a.iloc[:50,:]

,hbo2
870_dst,0.092667
860_dst,0.090546
850_dst,0.088788
880_dst,0.088777
980_dst,0.085435
830_dst,0.085097
900_dst,0.084500
810_dst,0.083051
820_dst,0.080626
940_dst,0.077260


In [58]:
corr = train.corr()
Survived = corr.loc[:,['ca']]
Survived = abs(Survived)
a = Survived.sort_values(by=['ca'],
                              ascending=False)
a = a[2:]
a

AttributeError: 'numpy.ndarray' object has no attribute 'corr'

In [32]:
corr = train.corr()
Survived = corr.loc[:,['na']]
Survived = abs(Survived)
a = Survived.sort_values(by=['na'],
                              ascending=False)
a = a[2:]
a

,na
910_dst,0.098124
730_dst,0.098076
720_dst,0.090701
920_dst,0.086032
890_dst,0.085807
...,...
hbo2,0.000726
830_src,0.000698
820_src,0.000611
840_dst,0.000323


In [2]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=160)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('랜덤포레스트 mae 는', mae)

랜덤포레스트 mae 는 1.4601283828124996


# GradientBoostingRegressor

In [9]:
y[:,0]

array([ 5.59,  0.  , 10.64, ...,  9.84,  6.38,  9.35])

In [10]:
a = []
for i in range(y.shape[1]) :
    a.append(y[:,i])

In [11]:
a

[array([ 5.59,  0.  , 10.64, ...,  9.84,  6.38,  9.35]),
 array([4.32, 2.83, 3.  , ..., 3.2 , 4.06, 4.34]),
 array([ 8.92,  7.25,  8.4 , ..., 10.45, 11.28,  9.73]),
 array([4.29, 4.64, 5.16, ..., 2.06, 4.03, 3.54])]

In [29]:
x_train, x_test, y_train, y_test = train_test_split(x,a[0], test_size=0.2, random_state=43)

model = GradientBoostingRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[1], test_size=0.2, random_state=43)
model = GradientBoostingRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[2], test_size=0.2, random_state=43)
model = GradientBoostingRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[3], test_size=0.2, random_state=43)
model = GradientBoostingRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

GradientBoostingRegressor mae 는 1.5838840045681222
GradientBoostingRegressor mae 는 0.7350616301783675
GradientBoostingRegressor mae 는 2.308885373741608
GradientBoostingRegressor mae 는 1.4267326043872253


In [13]:
x_train, x_test, y_train, y_test = train_test_split(x,a[0], test_size=0.2, random_state=43)

model = RandomForestRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[1], test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[2], test_size=0.2, random_state=43)
model = GradientBoostingRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[3], test_size=0.2, random_state=43)
model = GradientBoostingRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

In [20]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
pca.fit(y)
y1 = pca.transform(y)
x_train, x_test, y_train, y_test = train_test_split(x,y1, test_size=0.2, random_state=43)
model = GradientBoostingRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

C:\Users\bitcamp\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


랜덤포레스트 mae 는 1.837030398551365


# XGBRegressor()

In [54]:
y[:,0]

array([ 5.59,  0.  , 10.64, ...,  9.84,  6.38,  9.35])

In [56]:
x_train, x_test, y_train, y_test = train_test_split(x,y[:,0], test_size=0.2, random_state=43)

model = RandomForestRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y[:,1], test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) 
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y[:,2], test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y[:,3], test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

xgboost mae 는 1.28956995
xgboost mae 는 0.71051045
xgboost mae 는 2.187561183333333
xgboost mae 는 1.4065181833333333


In [55]:
x_train, x_test, y_train, y_test = train_test_split(x,y[:,0], test_size=0.2, random_state=43)

model = XGBRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y[:,1], test_size=0.2, random_state=43)
model = XGBRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test) 
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y[:,2], test_size=0.2, random_state=43)
model = XGBRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y[:,3], test_size=0.2, random_state=43)
model = XGBRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

xgboost mae 는 1.2829265556406977
xgboost mae 는 0.7349076338338851
xgboost mae 는 2.2790482931423184
xgboost mae 는 1.4711147229343653


In [32]:
x_train, x_test, y_train, y_test = train_test_split(x,a[0], test_size=0.2, random_state=43)

model = XGBRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[1], test_size=0.2, random_state=43)
model = XGBRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) 
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[2], test_size=0.2, random_state=43)
model = XGBRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[3], test_size=0.2, random_state=43)
model = XGBRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

xgboost mae 는 1.283430913808942
xgboost mae 는 0.7389190593361854
xgboost mae 는 2.2966433312320707
xgboost mae 는 1.4670573375791311


In [30]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
pca.fit(y)
y1 = pca.transform(y)
x_train, x_test, y_train, y_test = train_test_split(x,y1, test_size=0.2, random_state=43)
model = XGBRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

xgboost mae 는 1.8154817496832283


# RandomForestRegressor

In [31]:
x_train, x_test, y_train, y_test = train_test_split(x,a[0], test_size=0.2, random_state=43)

model = RandomForestRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('RandomForestRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[1], test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) 
mae = mean_absolute_error(y_pred, y_test)
print('RandomForestRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[2], test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('RandomForestRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[3], test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('RandomForestRegressor mae 는', mae)

xgboost mae 는 1.289325675
xgboost mae 는 0.7111221249999999
xgboost mae 는 2.187418975
xgboost mae 는 1.407558775


In [33]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=160)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) 
mae = mean_absolute_error(y_pred, y_test)
print('랜덤포레스트 mae 는', mae)

랜덤포레스트 mae 는 1.4622248984374993


In [37]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) 
mae = mean_absolute_error(y_pred, y_test)
print('랜덤포레스트 mae 는', mae)

랜덤포레스트 mae 는 1.7716422312499989


In [39]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=20)
model1 = RandomForestRegressor(n_estimators=250)
model1.fit(x_train,y_train)
y_pred = model1.predict(x_test) 
mae = mean_absolute_error(y_pred, y_test)
print('랜덤포레스트 mae 는', mae)

랜덤포레스트 mae 는 1.441592375


In [ ]:
pred = model1.predict(x_predict)
submission = pd.read_csv('sample_submission.csv', index_col=0)
submission['hhb'] = pred[:,0]
submission['hbo2'] = pred[:,1]
submission['ca'] = pred[:,2]
submission['na'] = pred[:,3]

('sample_submission.csv', index_col=0)

# 제출

In [42]:
# 제출
pred = model1.predict(x_predict)

pred = model1.predict(x_predict)
submission = pd.read_csv('sample_submission.csv', index_col=0)
submission['hhb'] = pred[:,0]
submission['hbo2'] = pred[:,1]
submission['ca'] = pred[:,2]
submission['na'] = pred[:,3]

submission.to_csv('submission0611_1.csv')

In [255]:
submission

,hhb,hbo2,ca,na
id,,,,
10000,7.330,3.861,7.608,2.890
10001,6.530,3.993,6.310,2.105
10002,11.439,4.057,9.335,3.524
10003,8.239,4.599,9.715,3.084
10004,8.279,3.615,9.837,4.130
...,...,...,...,...
19995,6.134,4.480,9.480,3.342
19996,7.258,4.027,9.550,3.275
19997,9.724,3.333,9.786,3.206
